In [ ]:
# Copyright (c) 2021 Intel Corporation.
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to
# deal in the Software without restriction, including without limitation the
# rights to use, copy, modify, merge, publish, distribute, sublicense, and/or
# sell copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM,OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS
# IN THE SOFTWARE.
"""Simple Notebook example UDF.
"""

import logging
import os
import cv2
import numpy as np


class Udf:
    """Example UDF
    """
    def __init__(self, udf_config):
        """Constructor

        :param udf_config: config of extra parameters to
                           be passed to udf if required
        :type udf_config: dict
        """
        logging.basicConfig(level=os.environ["PY_LOG_LEVEL"])
        self.log = logging.getLogger('JUPYTER_NOTEBOOK')
        self.log.info(f"In {__name__}...")

    def _write_text_on_frame(self, frame):
        """Write text on given frame

        :param frame: frame on which text is to be written
        :type frame: numpy
        """
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX

        # org
        org = (500, 500)

        # fontScale
        fontScale = 4

        # Blue color in BGR
        color = (0, 0, 255)

        # Line thickness of 2 px
        thickness = 3

        # Using cv2.putText() method
        image = cv2.putText(frame, 'JupyterNotebook', org, font,
                            fontScale, color, thickness, cv2.LINE_AA)
        
        return image

    def process(self, frame, metadata):
        """[summary]

        :param frame: frame blob
        :type frame: numpy.ndarray
        :param metadata: frame's metadata
        :type metadata: str
        :return:  (should the frame be dropped, has the frame been updated,
                   new metadata for the frame if any)
        :rtype: (bool, numpy.ndarray, str)
        """
        self.log.info(f"Add text...")

        height = int(metadata['height'])
        width = int(metadata['width'])
        channels = int(metadata['channels'])

        # Convert to Numpy array and reshape to frame
        encoding = None

        if 'encoding_type' and 'encoding_level' in metadata:
            encoding = {"type": metadata['encoding_type'],
                        "level": metadata['encoding_level']}
        # Convert to Numpy array and reshape to frame
        if isinstance(frame, list):
            # If multiple frames, select first frame
            frame = frame[0]
        frame = np.frombuffer(frame, dtype=np.uint8)
        if encoding is not None:
            frame = np.reshape(frame, (frame.shape))
            try:
                frame = cv2.imdecode(frame, 1)
            except cv2.error as ex:
                self.log.error("frame: {}, exception: {}".format(frame, ex))
        else:
            self.log.debug("Encoding not enabled...")
            frame = np.reshape(frame, (height, width, channels))

        # Write some text on the frame
        image = self._write_text_on_frame(frame)
        
        return False, image, None

